<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [86]:
import pandas as pd
import psycopg2
import plotly.express as px

In [87]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [88]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [9]:
# текст запроса
query_3_1 = f'''
    SELECT COUNT(id)
    FROM vacancies
    
'''

In [10]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

C:\Users\User\AppData\Local\Temp\ipykernel_16032\4045908245.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_1 = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [136]:
# текст запроса
query_3_2 = f'''
    SELECT DISTINCT COUNT(id)
    FROM employers
    
'''

In [137]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2

C:\Users\User\AppData\Local\Temp\ipykernel_7416\533540996.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [138]:
# текст запроса
query_3_3 = f'''
    SELECT DISTINCT COUNT(id)
    FROM areas
    
'''

In [139]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

C:\Users\User\AppData\Local\Temp\ipykernel_7416\1395976625.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [140]:
# текст запроса
query_3_4 = f'''
    SELECT DISTINCT COUNT(id)
    FROM industries
    
'''

In [141]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4

C:\Users\User\AppData\Local\Temp\ipykernel_7416\854197215.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,294


***

Выводы по предварительному анализу данных

Всего в базе находится 49197 уникальных вакансий в 294 сферах. База содержит вакансии 23,5 тысячи работодателей, которые распределены по 1362 регионам.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [154]:
# текст запроса
query_4_1 = f'''
    SELECT
        a.name AS "Регион",
        COUNT(distinct v.id) AS "Количество вакансий"
    FROM areas AS a
    JOIN vacancies AS v ON v.area_id = a.id
    GROUP BY 1
    ORDER BY 2 DESC LIMIT 5
    
'''

In [155]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

C:\Users\User\AppData\Local\Temp\ipykernel_7416\620650199.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Регион,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [152]:
# текст запроса
query_4_2 = f'''
    SELECT COUNT(distinct id) AS "Хотя бы одно поле ЗП заполнено"
    FROM
        (SELECT
            id,
            salary_from
        FROM vacancies
        WHERE salary_from IS NOT NULL
        
        UNION ALL
        
        SELECT id, salary_to
        FROM vacancies
        WHERE salary_to IS NOT NULL) x
        
'''

In [153]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2

C:\Users\User\AppData\Local\Temp\ipykernel_7416\3746829199.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Хотя бы одно поле ЗП заполнено
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [157]:
# текст запроса
query_4_3 = f'''
    SELECT
        ROUND(AVG(salary_from)) AS "Нижняя граница средней ЗП",
        ROUND(AVG(salary_to)) AS "Верхняя граница средней ЗП"
    FROM vacancies  
        
'''

In [158]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

C:\Users\User\AppData\Local\Temp\ipykernel_7416\3595259731.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Нижняя граница средней ЗП,Верхняя граница средней ЗП
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [6]:
# текст запроса
query_4_4 = f'''
    SELECT
        DISTINCT schedule AS "График",
        employment AS "Тип трудоустройства",
        COUNT(*) AS "Количество вакансий"
    FROM vacancies
    GROUP BY 1,2
    ORDER BY 3 DESC
        
'''

In [162]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

C:\Users\User\AppData\Local\Temp\ipykernel_7416\2028972612.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,График,Тип трудоустройства,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [163]:
# текст запроса
query_4_5 = f'''
    SELECT
        DISTINCT experience AS "Требуемый опыт работы",
        COUNT(*) AS "Количество вакансий"
    FROM vacancies
    GROUP BY 1
    ORDER BY 2
        
'''

In [164]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

C:\Users\User\AppData\Local\Temp\ipykernel_7416\210463167.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Требуемый опыт работы,Количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Выводы по детальному анализу вакансий

Больше всего вакансий доступно в самых крупных городах. В пятёрку лидеров входит Алматы - крупнейший город Казахстана. Это говорит о том, что речь идёт о рынке труда не только в России, а во всех странах, где популярен сервис hh.ru. При работе с этой базой необходимо учитывать региональные особенности рынка труда.

Только в половине вакансий (24072 из 49197) содержится хотя бы какая-то информация о зарплате. При этом средние значения для нижней и верхней границы зарплатной вилки составляют 71065 рублей и 110537 рублей

Наиболее типичным сочетанием типа трудоустройства и рабочего графика является полная занятость и полный рабочий день (35367 или около 70% всех вакансий в базе). Почти в 5 раз меньше (7802) - удалённая работа при полной занятости; на третьем месте - гибкий график при полной занятости (1593). А всего в базе содержится 18 различных сочетаний типа трудоустройства и рабочего графика.

Чаще всего работодатели ищут сотрудников с опытом 1-3 года (26152 - почти половина вакансий), почти треть вакансий - с опытом 3-6 лет. Сотрудники без опыта работы требуются в 15% вакансий, реже всего указывается требование более 6 лет. Можно предположить, что в подавляющем большинстве профессий наличие опыта желательно, но всего за пару-тройку лет среднестатистический соискатель приобретает опыт достаточный для полноценного выполнения работы.


***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [165]:
# текст запроса
query_5_1 = f'''
    SELECT
        e.name AS "Крупнейшие работодатели",
        COUNT(v.id) AS "Количество вакансий"
    FROM
        employers AS e
    JOIN
        vacancies AS v ON e.id = v.employer_id
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5

'''

In [166]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

C:\Users\User\AppData\Local\Temp\ipykernel_7416\1147638897.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Крупнейшие работодатели,Количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [4]:
# текст запроса
query_5_2 = f'''
    SELECT
        a.name AS "Регион",
        COUNT(DISTINCT e.id) AS "Количество работодателей",
        COUNT(v.id) AS "Количество вакансий"
    FROM areas AS a
    JOIN employers AS e ON a.id = e.area
    LEFT JOIN vacancies AS v ON a.id = v.area_id
    WHERE v.id IS NULL
    GROUP BY 1
    ORDER BY 3, 2 DESC
    
'''

In [5]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

C:\Users\User\AppData\Local\Temp\ipykernel_21712\2306256317.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_2 = pd.read_sql_query(query_5_2, connection)


,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
181,Миллерово,1,0
182,"Михайловка (Уфимский район, Башкортостан)",1,0
183,Можайск,1,0
184,Морозовск\n,1,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [7]:
# текст запроса
query_5_3 = f'''
    SELECT
        e.name AS "Работодатель",
        COUNT(DISTINCT v.area_id) AS "Количество регионов"
    FROM employers AS e
    LEFT JOIN vacancies AS v on e.id = v.employer_id
    GROUP BY 1
    ORDER BY 2 DESC

'''

In [8]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

C:\Users\User\AppData\Local\Temp\ipykernel_21712\133400711.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_3 = pd.read_sql_query(query_5_3, connection)


,Работодатель,Количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [9]:
# текст запроса
query_5_4 = f'''
    SELECT
        COUNT(e.id) AS "Сфера деятельности не указана"
    FROM employers e
    LEFT JOIN employers_industries ei on ei.employer_id = e.id
    WHERE ei.industry_id is null

'''

In [10]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

C:\Users\User\AppData\Local\Temp\ipykernel_21712\1045119256.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_4 = pd.read_sql_query(query_5_4, connection)


,Сфера деятельности не указана
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [11]:
# текст запроса
query_5_5 = f'''
    SELECT
        e.name AS  "Название компании",
        COUNT(ei.industry_id) AS "Количество сфер деятельности"
    FROM employers AS e
    JOIN employers_industries AS ei on e.id = ei.employer_id
    GROUP BY 1
    HAVING COUNT (ei.industry_id) = 4
    ORDER BY 1
    OFFSET 2
    LIMIT 1
    
'''

In [12]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

C:\Users\User\AppData\Local\Temp\ipykernel_21712\4025176131.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_5 = pd.read_sql_query(query_5_5, connection)


,Название компании,Количество сфер деятельности
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [13]:
# текст запроса
query_5_6 = f'''
    SELECT
        i.name AS "Cфера деятельности",
        COUNT(e.employer_id) AS "Количество работодателей"
    FROM employers_industries AS e
    LEFT JOIN industries AS i on e.industry_id = i.id
    WHERE i.name = 'Разработка программного обеспечения'
    GROUP BY 1

'''

In [14]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

C:\Users\User\AppData\Local\Temp\ipykernel_21712\2713946032.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_6 = pd.read_sql_query(query_5_6, connection)


,Cфера деятельности,Количество работодателей
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [15]:
# код для получения списка городов-милионников
import pandas as pd
import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)

df = pd.read_html(url)[1]
cities = tuple(df['Город'].tolist())
cities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [18]:
# текст запроса
query_5_7 = f'''
    (SELECT
        a.name "Город-миллионник",
        COUNT(v.id) "Количество вакансий"
    FROM areas AS a
    LEFT JOIN vacancies AS v ON a.id = v.area_id
    JOIN employers AS e ON v.employer_id = e.id
    WHERE e.name = 'Яндекс' AND a.name in {cities}
    GROUP BY 1
    ORDER BY 2)

    union all

    SELECT
        'Total',
        SUM(x.count)
    FROM (SELECT
            COUNT (v.id) AS count
        FROM areas AS a
        LEFT JOIN vacancies v ON a.id = v.area_id
        JOIN employers e ON v.employer_id = e.id
        WHERE e.name = 'Яндекс' AND a.name in {cities}) as x
    ORDER BY 2 DESC
'''

In [19]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

C:\Users\User\AppData\Local\Temp\ipykernel_21712\2595257991.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5_7 = pd.read_sql_query(query_5_7, connection)


,Город-миллионник,Количество вакансий
0,Total,485.0
1,Москва,54.0
2,Санкт-Петербург,42.0
3,Екатеринбург,39.0
4,Нижний Новгород,36.0
5,Новосибирск,35.0
6,Воронеж,32.0
7,Краснодар,30.0
8,Самара,26.0
9,Уфа,26.0


***

Выводы по анализу работодателей

Больше всего вакансий предлагает компания Яндекс (1933). Ростелеком, Тинькофф, Сбер и Газпром Нефть также в пятёрке лидеров.

Среди регионов без вакансий, но с большим количеством работодателей, преобладают наиболее крупные территориальные единицы - страны, а также крупные регионы России с большим количеством городов. Вероятно, это связано с тем, что работодатель указал территорией своей деятельности регион(ы), а вакансии публикует в конкретных городах.

Работодатель, предлагающий вакансии в максимальном количестве регионов — Яндекс (181 регион). На втором месте — Ростелеком (152 региона). На третьем месте некий предприниматель Поляков, который, судя по количеству регионов, где представлены его вакансии, разместил вакансии примерно в каждом субъекте РФ.

Почти четверть работодателей не указали сферу деятельности (по крайней мере, не сделали этого в том виде, в каком предлагает hh.ru)

С разработкой ПО связаны 3553 работодателя, разместивших вакансии на hh.ru

У Яндекса 485 вакансий в городах-миллионниках, причём количество вакансий не пропорционально количеству жителей. Так например, население Нижнего Новгорода или Новосибирска более чем в 10 раз меньше населения Москвы, но количество вакансий в каждом из этих городов составляет примерно 2/3 от московских.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [31]:
# текст запроса
query_6_1 = f'''
    SELECT COUNT(*) AS "Вакансии имеют отношение к данным"
    FROM vacancies
    WHERE 
        name ILIKE '%data%' 
        OR name Ilike '%данн%'

'''

In [32]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

C:\Users\User\AppData\Local\Temp\ipykernel_21712\1281472032.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_1 = pd.read_sql_query(query_6_1, connection)


,Вакансии имеют отношение к данным
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [52]:
# текст запроса
query_6_2 = f'''
    SELECT count(*) as "DS вакансии уровня Junior"
    FROM vacancies
    WHERE (name ilike '%data%scientist%' 
        OR name ilike '%data%science%'
        OR name ilike '%исследователь%данных%'
        OR (name like '%ML%' and name not ilike '%HTML%')
        OR name ilike '%machine%learning%'
        OR name ilike '%машинн%обучен%%')
        AND (name ilike '%junior%'
        OR experience = 'Нет опыта'
        OR employment = 'Стажировка')
    
'''

In [53]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

C:\Users\User\AppData\Local\Temp\ipykernel_21712\118162426.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_2 = pd.read_sql_query(query_6_2, connection)


,DS вакансии уровня Junior
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [39]:
# текст запроса
query_6_3 = f'''
    SELECT COUNT(*) as "Вакансии, где требуется SQL или postgres"
    FROM vacancies
    WHERE ((name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
        OR name ILIKE '%data%scientist%'
        OR name ILIKE '%data%science%'
        OR name ILIKE '%исследователь%данных%'
        OR name ILIKE '%machine%learning%'
        OR name ILIKE '%машинн%обучен%%')
        AND (key_skills ILIKE '%SQL%' OR key_skills ILIKE 'postgres')

'''

In [40]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

C:\Users\User\AppData\Local\Temp\ipykernel_21712\3310962512.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_3 = pd.read_sql_query(query_6_3, connection)


,"Вакансии, где требуется SQL или postgres"
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [37]:
# текст запроса
query_6_4 = f'''
    SELECT COUNT(*) as "Вакансии, где требуется Python"
    FROM vacancies
    WHERE ((name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
        OR name ILIKE '%data%scientist%'
        OR name ILIKE '%data%science%'
        OR name ILIKE '%исследователь%данных%'
        OR name ILIKE '%machine%learning%'
        OR name ILIKE '%машинн%обучен%%')
        AND (key_skills ilike '%python%')

'''

In [38]:
# результат запроса
df_6_4 = pd.read_sql_query(query_6_4, connection)
df_6_4

C:\Users\User\AppData\Local\Temp\ipykernel_21712\614304167.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_4 = pd.read_sql_query(query_6_4, connection)


,"Вакансии, где требуется Python"
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [44]:
# текст запроса
query_6_5 = f'''
    SELECT ROUND(AVG((LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')))::integer+1),2) "Среднее количество навыков для DS"
    FROM vacancies
    WHERE ((name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
        OR name ILIKE '%data%scientist%'
        OR name ILIKE '%data%science%'
        OR name ILIKE '%исследователь%данных%'
        OR name ILIKE '%machine%learning%'
        OR name ILIKE '%машинн%обучен%%')
        AND (key_skills IS NOT NULL)

'''

In [45]:
# результат запроса
df_6_5 = pd.read_sql_query(query_6_5, connection)
df_6_5

C:\Users\User\AppData\Local\Temp\ipykernel_21712\4089566367.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_5 = pd.read_sql_query(query_6_5, connection)


,Среднее количество навыков для DS
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [48]:
# текст запроса
query_6_6 = f'''
    SELECT
        experience AS "Опыт работы",
        ROUND(AVG(COALESCE((salary_to + salary_from)/2, salary_from, salary_to)),0) AS "Средняя зарплата"
    FROM vacancies
    WHERE ((name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
        OR name ILIKE '%data%scientist%'
        OR name ILIKE '%data%science%'
        OR name ILIKE '%исследователь%данных%'
        OR name ILIKE '%machine%learning%'
        OR name ILIKE '%машинн%обучен%%')
        AND (key_skills IS NOT NULL)
    GROUP BY 1
    ORDER BY 2 DESC
    '''


In [49]:
# результат запроса
df_6_6 = pd.read_sql_query(query_6_6, connection)
df_6_6

C:\Users\User\AppData\Local\Temp\ipykernel_21712\2297777544.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6_6 = pd.read_sql_query(query_6_6, connection)


,Опыт работы,Средняя зарплата
0,Более 6 лет,NaN
1,От 3 до 6 лет,243115.0
2,От 1 года до 3 лет,146704.0
3,Нет опыта,74643.0


***

Выводы по предметному анализу

1771 вакансия имеет отношение к данным

Из 483* вакансий для специалистов по DataScience лишь 51 подойдёт для начинающих (483 получено из задания 6.2. путём исключения фильтрации по критерию "для начинающих")

Среди DS-вакансий есть 201 вакансия, где в качестве ключевого навыка указан SQL или postgres и 351 вакансия, где в качестве ключевого навыка указан python

В среднем в требованиях для DS-вакансий работодатели указывают 6-7 навыков

DS-специалист без опыта работы в серднем может рассчитывать на 74,6 тыс. рублей, специалист с опытом работы от 1 года до 3 лет получают 146,7 тыс. руб., а со стажем более 3 лет — в среднем 243,1 тыс. руб.

# Общий вывод по проекту

Наиболее крупными работодателями являются IT-компании, банки, за ними следуют компании нефте-газового сектора.

Наибольшее количество вакансий сосредоточено в наиболее крупных городах в то время как в множестве мелких населённых пунктов вакансий нет совсем.

Работа специалистов по Data Science оценивается работодателями высоко и потому выглядит привлекательно, но работодатели предпочитают иметь дело с опытными специалистами. Вакансий для новичков совсем немного, поэтому на входе в профессию конкуренция высока.

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

# 7. Дополнительное исследование данных

7.1. Общий обзор: топ-10 отраслей по количеству вакансий в них

In [72]:
# текст запроса
query_7_1 = f'''

    SELECT
        i.name AS "Сфера деятельности компании",
        count (v.id) AS "Количество вакансий"  
    FROM vacancies AS v
    JOIN employers e ON v.employer_id = e.id
    JOIN (SELECT 
        DISTINCT employer_id,
        industry_id   
    FROM employers_industries) AS ei
        ON ei.employer_id = e.id
    JOIN industries i on ei.industry_id = i.id
    GROUP by 1
    ORDER BY 2 desc
    LIMIT 10
'''

In [73]:
# результат запроса
df_7_1 = pd.read_sql_query(query_7_1, connection)
df_7_1

C:\Users\User\AppData\Local\Temp\ipykernel_21712\1443944072.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Сфера деятельности компании,Количество вакансий
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
5,Кадровые агентства,1143
6,Консалтинговые услуги,1137
7,Интернет-провайдер,1095
8,Розничная сеть (продуктовая),1034
9,Фиксированная связь,1028


7.2. ТОП-10 отраслей по количеству вакансий для DS-специалистов

In [95]:
query_7_2 = f'''
    SELECT
        i.name "Название сферы деятельности",
        COUNT(v.id) "Количество DS-вакансий"
    FROM vacancies AS v
    JOIN employers_industries AS ei on v.employer_id = ei.employer_id
    JOIN industries AS i ON ei.industry_id = i.id
    WHERE ((v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%')
        OR v.name ILIKE '%data%scientist%'
        OR v.name ILIKE '%data%science%'
        OR v.name ILIKE '%исследователь%данных%'
        OR v.name ILIKE '%machine%learning%'
        OR v.name ILIKE '%машинн%обучен%%')
        AND (key_skills IS NOT NULL)
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10

'''

In [96]:
# результат запроса
df_7_2 = pd.read_sql_query(query_7_2, connection)
df_7_2

C:\Users\User\AppData\Local\Temp\ipykernel_21712\2016345237.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Название сферы деятельности,Количество DS-вакансий
0,Разработка программного обеспечения,144
1,"Системная интеграция, автоматизации технологи...",133
2,Банк,75
3,"Интернет-компания (поисковики, платежные систе...",73
4,"Маркетинговые, рекламные, BTL, дизайнерские, E...",22
5,"Управляющая, инвестиционная компания (управлен...",21
6,Мобильная связь,18
7,Консалтинговые услуги,16
8,Фиксированная связь,12
9,"Производство мультимедиа, контента, редакторск...",8


7.3. Какова зависимость заработной платы от опыта работы при найме DS-специалистов? Отличается ли это от ситуации на рынке труда в целом?

In [82]:
query_7_3 = f'''
    SELECT
        experience AS "Опыт работы",
        ROUND(AVG(salary)) AS "Заработная плата",
        'Вакансии в Data Science' as  "Тип вакансии"
    FROM (
        SELECT
            experience, 
            salary_from AS salary
        FROM vacancies AS v
        WHERE ((name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
            OR name ILIKE '%data%scientist%'
            OR name ILIKE '%data%science%'
            OR name ILIKE '%исследователь%данных%'
            OR name ILIKE '%machine%learning%'
            OR name ILIKE '%машинн%обучен%%')
            AND salary_from IS NOT NULL AND salary_to IS NULL
    
        UNION ALL
    
        SELECT
            experience, 
            salary_to
        FROM vacancies AS v
        WHERE ((name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
            OR name ILIKE '%data%scientist%'
            OR name ILIKE '%data%science%'
            OR name ILIKE '%исследователь%данных%'
            OR name ILIKE '%machine%learning%'
            OR name ILIKE '%машинн%обучен%%')
            AND salary_to IS NOT NULL AND salary_from IS NULL
                    
        UNION ALL
                    
        SELECT
            experience, 
            (salary_to + salary_from)/2
        FROM vacancies AS v
        WHERE ((name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
            OR name ILIKE '%data%scientist%'
            OR name ILIKE '%data%science%'
            OR name ILIKE '%исследователь%данных%'
            OR name ILIKE '%machine%learning%'
            OR name ILIKE '%машинн%обучен%%')
            AND salary_to IS NOT NULL AND salary_from IS NOT NULL
            ) as x
    GROUP BY 1
            
    UNION ALL
            
    SELECT 
        experience,
        ROUND(avg(salary)),
        'Все вакансии' as "Тип вакансии"
    FROM (
        SELECT
            experience, 
            salary_from AS salary
        FROM vacancies AS v
        WHERE salary_from IS NOT NULL AND salary_to IS NULL
                    
        UNION ALL
                    
        SELECT
            experience, 
            salary_to
        FROM vacancies AS v
        WHERE salary_to IS NOT NULL AND salary_from IS NULL
                    
        UNION ALL
                    
        SELECT
            experience, 
            (salary_to + salary_from)/2
        FROM vacancies AS v
        WHERE salary_to IS NOT NULL AND salary_from IS NOT NULL
            ) as x
    GROUP by 1    
            
'''

In [83]:
# результат запроса
df_7_3 = pd.read_sql_query(query_7_3, connection)
df_7_3 

C:\Users\User\AppData\Local\Temp\ipykernel_21712\21475310.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Опыт работы,Заработная плата,Тип вакансии
0,Нет опыта,74643.0,Вакансии в Data Science
1,От 1 года до 3 лет,139675.0,Вакансии в Data Science
2,От 3 до 6 лет,243115.0,Вакансии в Data Science
3,От 3 до 6 лет,133211.0,Все вакансии
4,От 1 года до 3 лет,76541.0,Все вакансии
5,Более 6 лет,166256.0,Все вакансии
6,Нет опыта,40472.0,Все вакансии


In [106]:
table = df_7_3.pivot(index='Тип вакансии', columns='Опыт работы', values = 'Заработная плата')

# перестановка столбцов для графика
cols = table.columns.tolist()
column_to_move = "Более 6 лет"
new_position = 4
cols.insert(new_position, cols.pop(cols.index(column_to_move)))
table = table[cols]
table

Опыт работы,Нет опыта,От 1 года до 3 лет,От 3 до 6 лет,Более 6 лет
Тип вакансии,,,,
Вакансии в Data Science,74643.0,139675.0,243115.0,NaN
Все вакансии,40472.0,76541.0,133211.0,166256.0


In [107]:
fig_7_3 = px.bar(table,
            color = "Опыт работы",
            barmode = 'group',
            title = 'Сравнение средних зарплат в Data Science со средними зарплатами в целом по рынку труда')
fig_7_3.show()
#fig_6.write_html("Diagrams/fig_6_Salary_MoveTrip.html")


### Выводы по дополнительному исследованию данных

Судя по количеству вакансий, в тех же отраслях, где сосредоточено наибольшее число вакансий в целом по рынку, также существует и потребность в специалистах по Data Science. В первую очередь это разработка программного обеспечения, системная интеграция, интернет-компании, банки и мобильная связь. Интересно, что такие высокотехнологичные отрасли как медицина, машиностроение, научные исследования, нефте-газовая промышленность не попали в ТОП-10 для DS-специалистов.

При сравнении средних заработных плат для DS-специалистов с данными по рынку труда в целом видно, что зарплаты DS-специалистов в среднем в 2 раза выше зарплат по рынку, причём в каждой из категорий опыта работы(кроме стажа более 6 лет - такое требование не предъявляется к DS-специалистам в базе hh.ru)